In [1]:
# do not touch this cell 
%load_ext jupyter_record


In [2]:
%%git_commit

import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack,vstack
import numpy as np


In [3]:
%%git_commit

training = pd.read_csv('../archive/Combined_Flights_2018.csv').sample(100000)
testing = pd.read_csv('../archive/Combined_Flights_2019.csv').sample(100000)




In [4]:
%%git_commit

training = training[training['DIVERTED'] == 0]
testing = testing[testing['CANCELLED'] == 0]



KeyError: 'DIVERTED'

In [ ]:
%%git_commit
training = training[['DAY_OF_WEEK', 'Airline', 'DEP_TIME_BLK', 'ORIGIN', 'DEST', 'DISTANCE', 'ARR_DEL15']]
testing = testing[['DAY_OF_WEEK', 'Airline', 'DEP_TIME_BLK', 'ORIGIN', 'DEST', 'DISTANCE', 'ARR_DEL15']]

